# Set up

In [ ]:
# update path with data dir
import sys
sys.path.append('../data/')
sys.path.append('../modelling/')

In [2]:
import player_data as player
from neural_nets import goals
import pandas as pd
from datetime import datetime
import numpy as np
import patsy

# Getting data

In [3]:
file_name = 'csvs/element_gameweek_2019-05-14.csv'

In [4]:
try:
    element_gameweek_df = pd.read_csv(file_name)
except:
    element_gameweek_df = player.get_element_gameweek_df()
    element_gameweek_df.to_csv(file_name)

In [17]:
current_event = 34
element_types = [2, 3, 4]
threshold_minutes = 1

In [18]:
element_gameweek_df =\
element_gameweek_df[
    (element_gameweek_df['event'] <= current_event)
    & (element_gameweek_df['element_type'].isin(element_types))
    & (element_gameweek_df['minutes'] >= threshold_minutes)
]

# Modelling

## Goals

In [19]:
goals_formula = 'goals_scored ~ was_home + C(element) + C(opposition_team)'

In [20]:
goals_response, goals_explanatory =\
patsy.dmatrices(goals_formula, element_gameweek_df, return_type='dataframe')

In [21]:
goals_model = goals.train_model(goals_explanatory, goals_response)

Epoch 1/20
8593/8593 [==============================] - 2s 194us/sample - loss: 0.1185
Epoch 2/20
8593/8593 [==============================] - 1s 140us/sample - loss: 0.1071
Epoch 3/20
8593/8593 [==============================] - 1s 138us/sample - loss: 0.1021
Epoch 4/20
8593/8593 [==============================] - 1s 138us/sample - loss: 0.0980
Epoch 5/20
8593/8593 [==============================] - 1s 138us/sample - loss: 0.0935
Epoch 6/20
8593/8593 [==============================] - 1s 140us/sample - loss: 0.0888
Epoch 7/20
8593/8593 [==============================] - 1s 139us/sample - loss: 0.0843
Epoch 8/20
8593/8593 [==============================] - 1s 138us/sample - loss: 0.0798
Epoch 9/20
8593/8593 [==============================] - 1s 138us/sample - loss: 0.0754
Epoch 10/20
8593/8593 [==============================] - 1s 138us/sample - loss: 0.0713
Epoch 11/20
8593/8593 [==============================] - 1s 139us/sample - loss: 0.0678
Epoch 12/20
8593/8593 [==================

# Predicting points

In [22]:
gameweek_fixtures_df = player.get_gameweek_fixtures_df(35, threshold_minutes, element_gameweek_df)

In [23]:
gameweek_fixtures_explanatory_df = player.get_gameweek_fixtures_explanatory_df(gameweek_fixtures_df)

## Goals

In [13]:
predicted_goals = goals.predict(gameweek_fixtures_explanatory_df, goals_model)

In [14]:
gameweek_fixtures_df['predicted_goals_scored'] = predicted_goals

In [15]:
gameweek_fixtures_df['predicted_goals_scored'].describe()

count    653.000000
mean       0.047924
std        0.279320
min       -0.655190
25%       -0.134705
50%        0.009160
75%        0.177072
max        1.471910
Name: predicted_goals_scored, dtype: float64

In [16]:
gameweek_fixtures_df[gameweek_fixtures_df['predicted_goals_scored'] > 0.5]

,own_team,opposition_team,element,was_home,element_type,predicted_goals_scored
339,19,11,402,1,2,0.665735
339,19,11,405,1,2,0.626309
339,19,11,413,1,3,0.556900
339,19,11,464,1,2,0.543391
339,19,11,489,1,3,0.505221
339,19,11,502,1,4,0.579362
339,19,11,560,1,4,0.736479
339,19,11,605,1,2,0.741343
341,15,16,324,1,3,0.715126
343,1,7,3,1,2,0.715976
